The workforce in new emerging technologies are using
disruptive approaches to win the best jobs, ones with high pay
and with good work life balance. This utility will provide real time analytics on how innovative the workforce is when trying to show that they are
right for the job, especially in the brand new subjects.This real time analystics solution will bring insights on most sought after skills, badges and certifications in the field of Software Engineering, Cybersecurity and Data
Science and showcase real time analytic of similar skills from
leading job portals focusing on what measures of talent are
highly valued by the tech industry today.







> STEPS

---








Below step will request you to upload keywords file named "PaceKeywords.xlsx" from your machine. Make sure that you have three tabs in excel named "Software_BadgeAndCert","Software_Skill","Software_Positions"

In [ ]:
from google.colab import files
!rm PaceKeywords.xlsx
uploaded = files.upload()

Saving PaceKeywords.xlsx to PaceKeywords.xlsx


Prerequisite: Below steps installs Selenium , imports python libraries and set up chrome driver for utility 

In [ ]:
pip install selenium


In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests
import matplotlib.pyplot as plt
import openpyxl as pxl

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

In [ ]:
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')



Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (116 kB/s)
Reading package lists... Done
Reading package lists... Done
Building

In [ ]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from datetime import date

Set Up header for HTTPRequest to be used for most of requests to Job portals

In [ ]:
header = {  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",  "X-Requested-With": "XMLHttpRequest"}



Function to loadSheet named "PaceKeywords.xlsx' into python dataframe. Function expects xlsx extension file. An attempt to load another extension file can give errors

In [ ]:
import io
def loadSheet(sheet_name):
  search_terms = pd.read_excel(io.BytesIO(uploaded['PaceKeywords.xlsx']),sheet_name=sheet_name)
  return search_terms


Function to analyse indeed job portal using HTTP GET Request. This function would require maintainence if underlying structure of site is changed or any additional restrictions are implemented by Indeed. 

In [ ]:
def indeed_analysis(sheet_name,search_terms_p):
  search_terms = search_terms_p
  indeed_list = []
  indeed_list_urls = []
  
  search_terms.iloc[:, 0] = search_terms.iloc[:, 0].str.strip('[]')

  for term in search_terms.iloc[:, 0]:
      term = term.replace(" ","%20")
      term = "\""+term+"\""
      if sheet_name == "":
        url = f'https://www.indeed.com/jobs?q={term}&l=United%20States&vjk=ea8b83a76166217a'
      else:
        sheet_namech = sheet_name.replace(" ","%20")
        sheet_namech = "\""+sheet_namech+"\""
        url = f'https://www.indeed.com/jobs?q={sheet_namech}%20{term}&l=United%20States&vjk=ea8b83a76166217a'
      indeed_list_urls.append(url)
      try:
          r = requests.get(url, headers=header, timeout=10)
          soup = bs(r.text, 'html.parser')
          count_str = soup.find('div', id="searchCountPages").get_text()
          numb = count_str.split()
          indeed_list.append(numb[-2])
      except Exception as e:
          #print(f'error: {e}')
          indeed_list.append(0)
  return [indeed_list,indeed_list_urls]


Function to analyse monster job portal using HTTP GET Request. This function would require maintainence if underlying structure of site is changed or any additional restrictions are implemented by Monster. 

In [ ]:
def monster_analysis(sheet_name,search_terms_p):
  monster_list = []
  monster_list_urls = []
  search_terms = search_terms_p
  search_terms.iloc[:, 0] = search_terms.iloc[:, 0].str.strip('[]')
  search_terms.iloc[:, 0] = search_terms.replace(' ', '-', regex=True)

  for term in search_terms.iloc[:, 0]:
      #print(term)
      term = term.replace(" ","%20")
      #term = "\""+term+"\""
      if sheet_name == "":
        url1 = f'https://www.monster.com/jobs/search/?q=__22{term}__22&where=USA'
        url2 = f'https://www.monster.com/jobs/search/?q=__27{term}__27&where=USA'
      else:
        sheet_namech = sheet_name.replace(" ","%20")
        url1 = f'https://www.monster.com/jobs/search/?q=__22{sheet_namech}__22-__22{term}__22&where=USA'
        url2 = f'https://www.monster.com/jobs/search/?q=__27{sheet_namech}__27-__27{term}__27&where=USA'
      monster_list_urls.append(url1)
      try:
          r = requests.get(url1, headers=header, timeout=5)
          soup = bs(r.text, 'html.parser')
          count_str = soup.find('h2', class_="figure").get_text()
        # print(count_str)
          numb = count_str.split()
          monster_count = numb[0].replace("(", "")
          monster_list.append(monster_count)
      except Exception as e:
      #   print(f'error: {e}')
          monster_list.append(0)
  return [monster_list,monster_list_urls]


Function to analyse Simply Hired job portal using HTTP GET Request. This function would require maintainence if underlying structure of site is changed or any additional restrictions are implemented by Simply Hired. 

In [ ]:
def simplyhired_analysis(sheet_name,search_terms_p):
  simply_list = []
  simply_list_url = []
  search_terms = search_terms_p
  search_terms.iloc[:, 0] = search_terms.iloc[:, 0].str.strip('[]')

  for term in search_terms.iloc[:, 0]:
      term = term.replace(" ","%20")
      term = "\""+term+"\""
      if sheet_name == "":
        url = f'https://www.simplyhired.com/search?q={term}&l=United+States'
      else:
        sheet_namech = sheet_name.replace(" ","%20")
        sheet_namech = "\""+sheet_namech+"\""
        url = f'https://www.simplyhired.com/search?q={sheet_namech}+{term}&l=United+States'
      simply_list_url.append(url)
      try:
          r = requests.get(url, headers=header, timeout=5)
          soup = bs(r.text, 'html.parser')
          count_str = soup.find('span', class_="CategoryPath-total").get_text()
          #print(count_str)
          simply_list.append(count_str.replace(',',''))
      except Exception as e:
          #print(f'error: {e}')
          simply_list.append(0)
  return [simply_list,simply_list_url]



Function to analyse Gartner job portal using Selenium API. This function would require maintainence if underlying structure of site is changed or any additional restrictions are implemented by Gartner. 

In [ ]:
def gartner_analysis(sheet_name,search_terms_p):
  gartner_list = []
  gartner_list_url = []
  search_terms=search_terms_p
  search_terms.iloc[:, 0] = search_terms.iloc[:, 0].str.strip('[]')

  for term in search_terms.iloc[:, 0]:
      term = term.replace(" ","%20")
      if sheet_name == "":
        url = f'https://jobs.gartner.com/search-jobs/{term}/United%20States/494/1/2/6252001/39x7599983215332/-98x5/50/2'
      else:
        sheet_namech = sheet_name.replace(" ","%20")
        url = f'https://jobs.gartner.com/search-jobs/{sheet_namech}%20{term}/United%20States/494/1/2/6252001/39x7599983215332/-98x5/50/2'
      gartner_list_url.append(url)
      try:
          wd = webdriver.Chrome('chromedriver',options=chrome_options)
          r = wd.get(url)
          #print(r)
          count_str = (wd.find_elements_by_class_name("results-count-heading"))[0].text
          #print(count_str)
          numb = count_str.split()
          gartner_list.append(numb[-2])
      except Exception as e:
          #print(f'error: {e}')
          gartner_list.append(0)
  return [gartner_list,gartner_list_url]
        


Function to analyse Dice job portal using Selenium API. This function would require maintainence if underlying structure of site is changed or any additional restrictions are implemented by Dice. 

In [ ]:
def dice_analysis(sheet_name,search_terms_p):
  dice_list = []
  dice_list_url = []
  search_terms = search_terms_p
  search_terms.iloc[0:2, 0] = search_terms.iloc[:, 0].str.strip('[]')
  #print("test")
  for term in search_terms.iloc[:, 0]:
      term = term.replace(" ","%20")
      term = "\""+term+"\""
      if sheet_name == "":
          url = f'https://www.dice.com/jobs?q={term}&location=United%20States'
      else:
         sheet_namech = sheet_name.replace(" ","%20")
         sheet_namech = "\""+sheet_namech+"\""
         url = f'https://www.dice.com/jobs?q={sheet_namech}%20{term}&location=United%20States'
      dice_list_url.append(url)
      timeout = 5
      try:
       wd = webdriver.Chrome('chromedriver',options=chrome_options)
       wd.get(url)
       element_present = EC.presence_of_element_located((By.ID, 'totalJobCount'))
       WebDriverWait(wd, timeout).until(element_present)
       count_str = (wd.find_element_by_id("totalJobCount")).get_attribute("innerText")
       numb = count_str.replace(",", "");
       dice_list.append(numb)
      except Exception as e:
        #print(f'error: {e}')
        dice_list.append(0)
  return [dice_list,dice_list_url]

       


Function to analyse Glassdoor job portal using Selenium API. This function would require maintainence if underlying structure of site is changed or any additional restrictions are implemented by Glassdoor. 

In [ ]:
def glassdoor_analysis(sheet_name,search_terms_p):
  glassdoor_list = []
  glassdoor_list_url = []
  search_terms = search_terms_p
  search_terms.iloc[0:2, 0] = search_terms.iloc[:, 0].str.strip('[]')
  #print("test")
  for term in search_terms.iloc[:, 0]:
      term = term.replace(" ","+")
      term = "\""+term+"\""
      if sheet_name == "":
          #url = f'https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=%22{term}%22&locT=N&locId=1&jobType=&context=Jobs&sc.keyword=%22{term}%22&dropdown=0
          url = f'https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword={term}&sc.keyword={term}&locT=N&locId=1'
      else:
         sheet_namech = sheet_name.replace(" ","+")
         sheet_namech = "\""+sheet_namech+"\""
         url = f'https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword={sheet_namech}+{term}&sc.keyword={sheet_namech}+{term}&locT=N&locId=1'
      glassdoor_list_url.append(url)
      timeout = 5
      try:
        wd = webdriver.Chrome('chromedriver',options=chrome_options)
        wd.get(url)
        count_str = (wd.find_element_by_xpath("//div[@class='hideHH css-19rczgc e15r6eig0']")).get_attribute("innerText")
        numb_split = count_str.split()
        numb = numb_split[0].replace(",", "")
        glassdoor_list.append(numb)
      except Exception as e:
        #print(f'error: {e}')
        glassdoor_list.append(0)
  return [glassdoor_list,glassdoor_list_url]

       


Function to analyse careerbuilder job portal using Selenium API. This function would require maintainence if underlying structure of site is changed or any additional restrictions are implemented by careerbuilder.

In [ ]:
def careerbuilderjobs_analysis(sheet_name,search_terms_p):
  careerbuilderjobs_list = []
  careerbuilderjobs_list_url = []
  search_terms = search_terms_p
  search_terms.iloc[0:2, 0] = search_terms.iloc[:, 0].str.strip('[]')
  #print("test")
  for term in search_terms.iloc[:, 0]:
      term = term.replace(" ","%20")
      term = "\""+term+"\""
      if sheet_name == "":
        url = f'https://www.careerbuilder.com/jobs?utf8=%E2%9C%93&keywords={term}&location=United+States'
      else:
        sheet_namech = sheet_name.replace(" ","%20")
        sheet_namech = "\""+sheet_namech+"\"" 
        url = f'https://www.careerbuilder.com/jobs?utf8=%E2%9C%93&keywords={sheet_namech}+{term}&location=United+States'
      careerbuilderjobs_list_url.append(url)
      timeout = 5
      try:
        wd = webdriver.Chrome('chromedriver',options=chrome_options)
        wd.get(url)
        count_str = (wd.find_element_by_xpath("//h1[@class='fz1rem']")).get_attribute("innerText")
        numb_split = count_str.split()
        for i in numb_split:
          numb = i.replace(",", "")
          if numb.isdigit():
            careerbuilderjobs_list.append(numb)
            break
      except Exception as e:
        #print(f'error: {e}')
        careerbuilderjobs_list.append(0)
  return [careerbuilderjobs_list,careerbuilderjobs_list_url]

       


Function to analyse flexjobs job portal using Selenium API. This function would require maintainence if underlying structure of site is changed or any additional restrictions are implemented by flexjobs.

In [ ]:
def flexjobs_analysis(sheet_name,search_terms_p):
  flexjobs_list = []
  flexjobs_list_url=[]
  search_terms = search_terms_p
  search_terms.iloc[0:2, 0] = search_terms.iloc[:, 0].str.strip('[]')
  #print("test")
  for term in search_terms.iloc[:, 0]:
      term = term.replace(" ","%20")
      term = "\""+term+"\""
      if sheet_name == "":
        url = f'https://www.flexjobs.com/search?search=&search={term}&location=United+States'
      else:
        sheet_namech = sheet_name.replace(" ","%20")
        sheet_namech = "\""+sheet_namech+"\""
        url = f'https://www.flexjobs.com/search?search=&search={sheet_namech}+{term}&location=United+States' 
      flexjobs_list_url.append(url)
      timeout = 5
      try:
        wd = webdriver.Chrome('chromedriver',options=chrome_options)
        wd.get(url)
        count_str = (wd.find_element_by_xpath("//h4[@style='margin:0;font-size:14px;']")).get_attribute("innerText")
        numb_split = count_str.split("of")[1].split("for")
        numb = numb_split[0].replace(",", "")
        flexjobs_list.append(numb)
      except Exception as e:
        #print(f'error: {e}')
        flexjobs_list.append(0)
  return [flexjobs_list,flexjobs_list_url]

       


Function to analyse find.jobs job portal using Selenium. This function would require maintainence if underlying structure of site is changed or any additional restrictions are implemented by Find.jobs.

In [ ]:
def findjobs_analysis(sheet_name,search_terms_p):
  usjobs_list = []
  usjobs_list_url =[]
  search_terms = search_terms_p
  search_terms.iloc[0:2, 0] = search_terms.iloc[:, 0].str.strip('[]')
  #print("test")
  for term in search_terms.iloc[:, 0]:
      term = term.replace(" ","%20")
      term = "\""+term+"\""
      if sheet_name == "":
        url = f'https://find.jobs/jobs-near-me/job-search-results/kw-{term}/co-226/'
      else:
        sheet_namech = sheet_name.replace(" ","%20")
        sheet_namech = "\""+sheet_namech+"\""
        url = f'https://find.jobs/jobs-near-me/job-search-results/kw-{sheet_namech}-{term}/co-226/'
      usjobs_list_url.append(url)
      timeout = 5
      try:
        wd = webdriver.Chrome('chromedriver',options=chrome_options)
        wd.get(url)
        count_str = (wd.find_element_by_xpath("//p[@class='results-count']")).get_attribute("innerText")
        numb_split = count_str.split(" ")[3]
        numb = numb_split.replace(",", "")
        usjobs_list.append(numb)
      except Exception as e:
        #print(f'error: {e}')
        usjobs_list.append(0)
  return [usjobs_list,usjobs_list_url]

       


Function to analyse USJobsGov job portal using REST API exposed by portal. This function would require maintainence if underlying API call exposed by site is changed.

In [ ]:
import requests
import json

def jprint(obj):
    # create a formatted string of the Python JSON object
    text = json.dumps(obj, sort_keys=True, indent=4)
    text_dict = json.loads(text)
    return text_dict['SearchResult']['SearchResultCountAll']

def usjobsgov_analysis(sheet_name,search_terms_p):
    hh = {"Host": "data.usajobs.gov","User-Agent": "anju.jain@pace.edu", "Authorization-Key": "K675H4vq5k2oqeu4XSrWiNNgMtC1gKw1t0taBgzMfD0=" }
    usjobsgov_list = []
    usjobsgov_list_url =[]
    search_terms = search_terms_p
    search_terms.iloc[0:2, 0] = search_terms.iloc[:, 0].str.strip('[]')
    #print("test")
    for term in search_terms.iloc[:, 0]:
      term = term.replace(" ","%20")
      term = "\""+term+"\""
      if sheet_name == "":
          url = f"https://data.usajobs.gov/api/search?Keyword={term}&PositionLocation['CountryCode']=United%20States"
          response = requests.get(url, headers=hh)
      else:
          sheet_namech = sheet_name.replace(" ","%20")
          sheet_namech = "\""+sheet_namech+"\""
          url = f"https://data.usajobs.gov/api/search?Keyword={sheet_namech},{Railroad}&PositionLocation['CountryCode']=United%20States"
          response = requests.get(url, headers=hh)
      usjobsgov_list_url.append(url)
      numb = jprint(response.json())
    #print(numb)
      usjobsgov_list.append(numb)
    return [usjobsgov_list,usjobsgov_list_url]




Below step invokes loadSheet function and reads tab named "Software_Skill". Any change to tab name in input file could result into errors or not loading expected data.

In [ ]:
search_terms_skills = loadSheet("Software_Skill")
search_terms_skills

,Keywords
0,Java
1,Javascript
2,SQL
3,Angular
4,Python
5,HTML
6,Git
7,React
8,aws
9,C++


Below step invokes loadSheet function and reads tab named "Software_Positions". Any change to tab name in input file could result into errors or not loading expected data.

In [ ]:
search_terms_positions = loadSheet("Software_Positions")
search_terms_positions

,Keywords
0,Software Engineer
1,Software Analyst
2,Software Architect
3,Software Administrator
4,Software Manager
5,Software Auditor
6,Software Consultant
7,Software Manager
8,Software Systems Engineer
9,Software Network Engineer


Below step invokes loadSheet function and reads tab named "Software_BadgeAndCert". Any change to tab name in input file could result into errors or not loading expected data.

In [ ]:
search_terms_BC = loadSheet("Software_BadgeAndCert")
search_terms_BC

,Keywords
0,Sun Certified Java Developer
1,Sun Certified Web Developer
2,Oracle Certified Associate
3,Oracle Certified Professional
4,Oracle Certified Master
5,Oracle Certfied Expert
6,Oracle Certified Junior Associate
7,Certified Entry Level Python Programmer
8,Certfied Associate in Python Programming
9,Certified Professional in Python


In [ ]:
#glassdoor_skills_listOflist = glassdoor_analysis("",search_terms_skills)    
#glassdoor_skills_list=glassdoor_skills_listOflist[0]
#glassdoor_skills_list_url = glassdoor_skills_listOflist[1]

Below calls executes functions against job portals for skills as read from input file

In [ ]:
indeed_Skills_listOflist = indeed_analysis("",search_terms_skills)
indeed_Skills_list=indeed_Skills_listOflist[0]
indeed_Skills_list_url = indeed_Skills_listOflist[1]

monster_skills_listOflist = monster_analysis("",search_terms_skills)    
monster_skills_list=monster_skills_listOflist[0]
monster_skills_list_url = monster_skills_listOflist[1]

careerbuilderjobs_skills_listOflist = careerbuilderjobs_analysis("",search_terms_skills)    
careerbuilderjobs_skills_list=careerbuilderjobs_skills_listOflist[0]
careerbuilderjobs_skills_list_url = careerbuilderjobs_skills_listOflist[1]
careerbuilderjobs_skills_list

flexjobs_Skill_listOflist = flexjobs_analysis("",search_terms_skills)    
flexjobs_Skill_list=flexjobs_Skill_listOflist[0]
flexjobs_Skill_list_url = flexjobs_Skill_listOflist[1]

simply_skills_listOflist = simplyhired_analysis("",search_terms_skills)    
simply_skills_list=simply_skills_listOflist[0]
simply_skills_list_url = simply_skills_listOflist[1]

gartner_skills_listOflist = gartner_analysis("",search_terms_skills)    
gartner_skills_list=gartner_skills_listOflist[0]
gartner_skills_list_url = gartner_skills_listOflist[1]

dice_skills_listOflist = dice_analysis("",search_terms_skills)    
dice_skills_list=dice_skills_listOflist[0]
dice_skills_list_url = dice_skills_listOflist[1]

usjobsgov_listOflist = usjobsgov_analysis("",search_terms_skills)
usjobsgov_list=usjobsgov_listOflist[0]
usjobsgov_list_url = usjobsgov_listOflist[1]

findjobs_skills_listOflist = findjobs_analysis("",search_terms_skills)    
findjobs_skills_list=findjobs_skills_listOflist[0]
findjobs_skills_list_url = findjobs_skills_listOflist[1]


Below steps prepare output file from results recieved of counts from portals and write to excel file.

In [ ]:
df_skills = pd.DataFrame(indeed_Skills_list, index=search_terms_skills.iloc[:,0], columns=['Indeed'])
df_skills_url = pd.DataFrame(indeed_Skills_list_url, index=search_terms_skills.iloc[:,0], columns=['Indeed'])


In [ ]:
df_skills.rename(index={'':'CS_Skill'},inplace=True)
df_skills_url.rename(index={'':'CS_Skill'},inplace=True)


In [ ]:
df_skills['Gartner'] = gartner_skills_list
df_skills['Dice'] = dice_skills_list
df_skills['Monster'] = monster_skills_list
df_skills['SimplyHired'] = simply_skills_list
#df_skills['Glassdoor'] = glassdoor_skills_list
df_skills['flexjobs'] = flexjobs_Skill_list
df_skills['CareerBuilder'] = careerbuilderjobs_skills_list
df_skills['findJobs'] = findjobs_skills_list
df_skills['USJobsGov'] = usjobsgov_list
df_skills['Date'] = date.today()
new_order = [-1,0,1,2,3,4,5,6,7,8]
df_skills = df_skills[df_skills.columns[new_order]]
with pd.ExcelWriter('PortalsCountsSoftwareEngg.xlsx') as writer:  
    df_skills.to_excel(writer, sheet_name='SkillsAnalysis')

df_skills_url['Gartner'] = gartner_skills_list_url
df_skills_url['Dice'] = dice_skills_list_url
df_skills_url['Monster'] = monster_skills_list_url
df_skills_url['SimplyHired'] = simply_skills_list_url
#df_skills_url['Glassdoor'] = glassdoor_skills_list
df_skills_url['flexjobs'] = flexjobs_Skill_list_url
df_skills_url['CareerBuilder'] = careerbuilderjobs_skills_list_url
df_skills_url['findJobs'] = findjobs_skills_list_url
df_skills_url['USJobsGov'] = findjobs_skills_list_url
df_skills_url['Date'] = date.today()
df_skills_url = df_skills_url[df_skills_url.columns[new_order]]

with pd.ExcelWriter('PortalsCountsSoftwareEnggURL.xlsx') as writer:  
    df_skills_url.to_excel(writer, sheet_name='SkillsAnalysisURL')



In [ ]:
df_skills

,Date,Indeed,Gartner,Dice,Monster,SimplyHired,flexjobs,CareerBuilder,findJobs,USJobsGov
Keywords,,,,,,,,,,
Java,2020-10-02,"53,066",11,12089,33939,35854,1817,5000,2069,42
Javascript,2020-10-02,"38,357",9,8937,21757,28964,2404,5000,944,23
SQL,2020-10-02,"67,277",19,12745,39734,52711,2652,5000,1985,79
Angular,2020-10-02,"11,956",4,1707,6987,9060,520,3420,373,17
Python,2020-10-02,"60,278",17,8679,32137,43840,2295,5000,2149,25
HTML,2020-10-02,"40,275",6,3540,11674,29427,1412,5000,554,20
Git,2020-10-02,"19,448",5,3938,10619,15338,1015,5000,832,4
React,2020-10-02,"41,239",1,1371,17174,23062,1420,5000,672,167
aws,2020-10-02,"44,451",13,2818,23442,30662,2159,5000,1413,13


Below steps executes functions against job portals for skills per position as read from input file. Also, Prepare output file from results recieved of counts from portals for skills per first 3 positions( as in input excel) and write to excel file.

In [ ]:
df_skills_r = None
Ct = 1

for role in search_terms_positions.iloc[0:3,0]:
  roleFinal = role
  print(roleFinal)
  indeed_Skills_listOflist = indeed_analysis(roleFinal,search_terms_skills)
  indeed_Skills_list1=indeed_Skills_listOflist[0]
  indeed_Skills_list_url1 = indeed_Skills_listOflist[1]

  monster_skills_listOflist = monster_analysis(roleFinal,search_terms_skills)    
  monster_skills_list1=monster_skills_listOflist[0]
  monster_skills_list_url1 = monster_skills_listOflist[1]

  careerbuilderjobs_skills_listOflist = careerbuilderjobs_analysis(roleFinal,search_terms_skills)    
  careerbuilderjobs_skills_list1=careerbuilderjobs_skills_listOflist[0]
  careerbuilderjobs_skills_list_url1 = careerbuilderjobs_skills_listOflist[1]

  flexjobs_Skill_listOflist = flexjobs_analysis(roleFinal,search_terms_skills)    
  flexjobs_Skill_list1=flexjobs_Skill_listOflist[0]
  flexjobs_Skill_list_url1 = flexjobs_Skill_listOflist[1]

  simply_skills_listOflist = simplyhired_analysis(roleFinal,search_terms_skills)    
  simply_skills_list1=simply_skills_listOflist[0]
  simply_skills_list_url1 = simply_skills_listOflist[1]

  dice_skills_listOflist = dice_analysis(roleFinal,search_terms_skills)    
  dice_skills_list1=dice_skills_listOflist[0]
  dice_skills_list_url1 = dice_skills_listOflist[1]

  findjobs_skills_listOflist = findjobs_analysis(roleFinal,search_terms_skills)    
  findjobs_skills_list1=findjobs_skills_listOflist[0]
  findjobs_skills_list_url1 = findjobs_skills_listOflist[1]

  usjobsgov_skills_listOflist = usjobsgov_analysis("",search_terms_skills)    
  usjobsgov_skills_list1=usjobsgov_skills_listOflist[0]
  usjobsgov_skills_list_url1 = usjobsgov_skills_listOflist[1]

  #if df_skills_r is None:
  df_skills_r = pd.DataFrame(indeed_Skills_list1, index=search_terms_skills.iloc[:,0], columns=['Indeed'])
  df_skills_r.rename(index={'':'Data_Skill_r'},inplace=True)
  # GARTNER Double Qutoes with postion and skill does not work . df_skills_r['Gartner'] = gartner_skills_list1
  df_skills_r['Dice'] = dice_skills_list1
  df_skills_r['Monster'] = monster_skills_list1
  df_skills_r['SimplyHired'] = simply_skills_list1
  #df_skills_r['Glassdoor'] = glassdoor_skills_list1
  df_skills_r['flexjobs'] = flexjobs_Skill_list1
  df_skills_r['CareerBuilder'] = careerbuilderjobs_skills_list1
  df_skills_r['findJobs'] = findjobs_skills_list1
  df_skills_r['UsjobsGov'] = usjobsgov_skills_list1
  df_skills_r['Date'] = date.today()
  new_order = [-1,0,1,2,3,4,5,6,7]
  df_skills_r = df_skills_r[df_skills_r.columns[new_order]]
  
  df_skills_r_url = pd.DataFrame(indeed_Skills_list_url1, index=search_terms_skills.iloc[:,0], columns=['Indeed'])
  df_skills_r_url.rename(index={'':'Skill_r'},inplace=True)
  # GARTNER Double Qutoes with postion and skill does not work .df_skills_url['Gartner'] = gartner_skills_list_url1
  df_skills_r_url['Dice'] = dice_skills_list_url1
  df_skills_r_url['Monster'] = monster_skills_list_url1
  df_skills_r_url['SimplyHired'] = simply_skills_list_url1
  #df_skills_url['Glassdoor'] = glassdoor_skills_list
  df_skills_r_url['flexjobs'] = flexjobs_Skill_list_url1
  df_skills_r_url['CareerBuilder'] = careerbuilderjobs_skills_list_url1
  df_skills_r_url['findJobs'] = findjobs_skills_list_url1
  df_skills_r_url['UsjobsGov'] = usjobsgov_skills_list_url1
  df_skills_r_url['Date'] = date.today()
  df_skills_r_url = df_skills_r_url[df_skills_r_url.columns[new_order]]
  with pd.ExcelWriter('PortalsCountsSoftwareEngg.xlsx',mode='a') as writer:  
       df_skills_r.to_excel(writer, sheet_name=roleFinal+'_S')
  with pd.ExcelWriter('PortalsCountsSoftwareEnggURL.xlsx',mode='a') as writer:  
       df_skills_r_url.to_excel(writer, sheet_name=roleFinal+'_URL')
  Ct = Ct+1
 

Software Engineer
Software Analyst
Software Architect


Below steps executes functions against job portals for position as read from input file.

In [ ]:
careerbuilderjobs_positions_listOflist = careerbuilderjobs_analysis("",search_terms_positions)    
careerbuilderjobs_positions_list=careerbuilderjobs_positions_listOflist[0]
careerbuilderjobs_positions_list_url = careerbuilderjobs_positions_listOflist[1]

flexjobs_positions_listOflist = flexjobs_analysis("",search_terms_positions)    
flexjobs_positions_list=flexjobs_positions_listOflist[0]
flexjobs_positions_list_url = flexjobs_positions_listOflist[1]

#glassdoor_positions_list = glassdoor_analysis("Data",search_terms_positions)
findJobs_positions_listOflist = findjobs_analysis("",search_terms_positions)
findjobs_positions_list=findJobs_positions_listOflist[0]
findjobs_positions_list_url = findJobs_positions_listOflist[1]

gartner_positions_listOflist = gartner_analysis("",search_terms_positions)    
gartner_positions_list=gartner_positions_listOflist[0]
gartner_positions_list_url = gartner_positions_listOflist[1]

indeed_positions_listOflist = indeed_analysis("",search_terms_positions)
indeed_positions_list=indeed_positions_listOflist[0]
indeed_positions_list_url = indeed_positions_listOflist[1]

monster_positions_listOflist = monster_analysis("",search_terms_positions)    
monster_positions_list=monster_positions_listOflist[0]
monster_positions_list_url = monster_positions_listOflist[1]

careerbuilderjobs_positions_listOflist = careerbuilderjobs_analysis("",search_terms_positions)    
careerbuilderjobs_positions_list=careerbuilderjobs_positions_listOflist[0]
careerbuilderjobs_positions_list_url = careerbuilderjobs_positions_listOflist[1]

simply_positions_listOflist = simplyhired_analysis("",search_terms_positions)    
simply_positions_list=simply_positions_listOflist[0]
simply_positions_list_url = simply_positions_listOflist[1]

dice_positions_listOflist = dice_analysis("",search_terms_positions)    
dice_positions_list=dice_positions_listOflist[0]
dice_positions_list_url = dice_positions_listOflist[1]

findjobs_positions_listOflist = findjobs_analysis("",search_terms_positions)    
findjobs_positions_list=findjobs_positions_listOflist[0]
findjobs_positions_list_url = findjobs_positions_listOflist[1]

usjobsgov_positions_listOflist = usjobsgov_analysis("",search_terms_positions)    
usjobsgov_positions_list=usjobsgov_positions_listOflist[0]
usjobsgov_positions_list_url = usjobsgov_positions_listOflist[1]

Below steps prepare output file from results recieved of counts from portals for positions and write to excel file.

In [ ]:
df_positions = pd.DataFrame(indeed_positions_list, index=search_terms_positions.iloc[:, 0], columns=['Indeed'])
df_positions.rename(index={'':'Data_positions'},inplace=True)

df_positions_url = pd.DataFrame(indeed_positions_list_url, index=search_terms_positions.iloc[:, 0], columns=['Indeed'])
df_positions_url.rename(index={'':'Data_positions url'},inplace=True)

df_positions['Gartner'] = gartner_positions_list
df_positions['Dice'] = dice_positions_list
df_positions['Monster'] = monster_positions_list
df_positions['SimplyHired'] = simply_positions_list
#df_positions['Glassdoor'] = glassdoor_positions_list
df_positions['flexjobs'] = flexjobs_positions_list
df_positions['CareerBuilder'] = careerbuilderjobs_positions_list
df_positions['findJobs'] = findjobs_positions_list
df_positions['UsjobsGov'] = usjobsgov_positions_list
df_positions['Date'] = date.today()
new_order = [-1,0,1,2,3,4,5,6,7,8]
df_positions = df_positions[df_positions.columns[new_order]]


df_positions_url['Gartner'] = gartner_positions_list_url
df_positions_url['Dice'] = dice_positions_list_url
df_positions_url['Monster'] = monster_positions_list_url
df_positions_url['SimplyHired'] = simply_positions_list_url
#df_positions_url['Glassdoor'] = glassdoor_positions_list
df_positions_url['flexjobs'] = flexjobs_positions_list_url
df_positions_url['CareerBuilder'] = careerbuilderjobs_positions_list_url
df_positions_url['Usjobs'] = findjobs_positions_list_url
df_positions_url['UsjobsGov'] = usjobsgov_positions_list_url
df_positions_url['Date'] = date.today()
df_positions_url = df_positions_url[df_positions_url.columns[new_order]]

with pd.ExcelWriter('PortalsCountsSoftwareEngg.xlsx',mode='a') as writer:  
    df_positions.to_excel(writer, sheet_name='SoftwareEnggPositionsAnalysis')
with pd.ExcelWriter('PortalsCountsSoftwareEnggURL.xlsx',mode='a') as writer:  
    df_positions_url.to_excel(writer, sheet_name='SoftwareEnggPositionsAnalysisUrl')



/usr/local/lib/python3.6/dist-packages/openpyxl/workbook/child.py:102: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


In [ ]:
df_positions

,Date,Indeed,Gartner,Dice,Monster,SimplyHired,flexjobs,CareerBuilder,findJobs,UsjobsGov
Keywords,,,,,,,,,,
Software-Engineer,2020-10-02,"30,256",27,4348,27822,19973,2197,4846,989,4
Software-Analyst,2020-10-02,99,39,137,727,84,3,17,1,0
Software-Architect,2020-10-02,940,15,400,807,742,37,197,26,0
Software-Administrator,2020-10-02,37,13,2,25,34,0,7,1,0
Software-Manager,2020-10-02,94,108,3,55,81,0,12,11,0
Software-Auditor,2020-10-02,0,11,0,0,0,0,0,0,0
Software-Consultant,2020-10-02,138,67,3,30,0,58,22,1,0
Software-Manager-,2020-10-02,94,108,3,55,81,0,12,11,0
Software-Systems-Engineer,2020-10-02,176,38,31,135,145,7,0,1,0


Below steps executes functions against job portals for Badges And Certificates as read from input file.

In [ ]:
careerbuilder_BC_listOflist = careerbuilderjobs_analysis("",search_terms_BC)
careerbuilder_BC_list=careerbuilder_BC_listOflist[0]
careerbuilder_BC_list_url = careerbuilder_BC_listOflist[1]

flexjobs_BC_listOflist = flexjobs_analysis("",search_terms_BC)
flexjobs_BC_list=flexjobs_BC_listOflist[0]
flexjobs_BC_list_url = flexjobs_BC_listOflist[1]

indeed_BC_listOflist = indeed_analysis("",search_terms_BC)
indeed_BC_list=indeed_BC_listOflist[0]
indeed_BC_list_url = indeed_BC_listOflist[1]

indeed_BC_listOflist = indeed_analysis("",search_terms_BC)
indeed_BC_list=indeed_BC_listOflist[0]
indeed_BC_list_url = indeed_BC_listOflist[1]

monster_BC_listOflist = monster_analysis("",search_terms_BC)    
monster_BC_list=monster_BC_listOflist[0]
monster_BC_list_url = monster_BC_listOflist[1]

gartner_BC_listOflist = gartner_analysis("",search_terms_BC)    
gartner_BC_list=gartner_BC_listOflist[0]
gartner_BC_list_url = gartner_BC_listOflist[1]

simply_BC_listOflist = simplyhired_analysis("",search_terms_BC)    
simply_BC_list=simply_BC_listOflist[0]
simply_BC_list_url = simply_BC_listOflist[1]

dice_BC_listOflist = dice_analysis("",search_terms_BC)    
dice_BC_list=dice_BC_listOflist[0]
dice_BC_list_url = dice_BC_listOflist[1]

findjobs_BC_listOflist = findjobs_analysis("",search_terms_BC)    
findjobs_BC_list=findjobs_BC_listOflist[0]
findjobs_BC_list_url = findjobs_BC_listOflist[1]

usjobsgov_BC_listOflist = usjobsgov_analysis("",search_terms_BC)    
usjobsgov_BC_list=usjobsgov_BC_listOflist[0]
usjobsgov_BC_list_url = usjobsgov_BC_listOflist[1]


Below steps prepare output file from results recieved of counts from portals for Badegs and certificates and write to excel file.

In [ ]:
df_BC = pd.DataFrame(indeed_BC_list, index=search_terms_BC.iloc[:, 0], columns=['Indeed '])

df_BC.rename(index={'':'Data_BC'},inplace=True)

df_BC_url = pd.DataFrame(indeed_BC_list_url, index=search_terms_BC.iloc[:, 0], columns=['Indeed '])

df_BC_url.rename(index={'':'Data_BC_url'},inplace=True)

df_BC['Dice'] = dice_BC_list
df_BC['Gartner'] = gartner_BC_list
df_BC['Monster'] = monster_BC_list
df_BC['SimplyHired'] = simply_BC_list
#df_BC['Glassdoor'] = glassdoor_BC_list
df_BC['flexjobs'] = flexjobs_BC_list
df_BC['CareerBuilder'] = careerbuilder_BC_list
df_BC['findJobs'] = findjobs_BC_list
df_BC['Usjobs'] = usjobsgov_BC_list
df_BC['Date'] = date.today()
new_order = [-1,0,1,2,3,4,5,6,7,8]
df_BC = df_BC[df_BC.columns[new_order]]

df_BC_url['Gartner'] = gartner_BC_list_url
df_BC_url['Dice'] = dice_BC_list_url
df_BC_url['Monster'] = monster_BC_list_url
df_BC_url['SimplyHired'] = simply_BC_list_url
#df_BC_url['Glassdoor'] = glassdoor_BC_list
df_BC_url['flexjobs'] = flexjobs_BC_list_url
df_BC_url['CareerBuilder'] = careerbuilder_BC_list_url
df_BC_url['findJobs'] = findjobs_BC_list_url
df_BC_url['Usjobs'] = usjobsgov_BC_list_url
df_BC_url['Date'] = date.today()
df_BC_url = df_BC_url[df_BC_url.columns[new_order]]

with pd.ExcelWriter('PortalsCountsSoftwareEngg.xlsx',mode='a') as writer:  
    df_BC.to_excel(writer, sheet_name='SoftwareEnggBCAnalysis')

with pd.ExcelWriter('PortalsCountsSoftwareEnggURL.xlsx',mode='a') as writer:  
    df_BC_url.to_excel(writer, sheet_name='SoftwareEnggBCAnalysisUrl')

/usr/local/lib/python3.6/dist-packages/openpyxl/workbook/child.py:102: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


Download xlsx in computer

In [ ]:
from google.colab import files
files.download('PortalsCountsSoftwareEngg.xlsx')

from google.colab import files
files.download('PortalsCountsSoftwareEnggURL.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>